In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tf_agents
import os
import random
from collections import defaultdict
from tqdm import tqdm
from absl import logging

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment

from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.policies.policy_saver import PolicySaver
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.train.utils import train_utils
from tf_agents.metrics import tf_metrics

import tensorflow_probability as tfp
from tf_agents.utils import nest_utils

In [3]:
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data")

### Data Load
##### 실제 분석에 사용하는 데이터 셋은 ratings.dat 파일이며, 이를 pandas DataFrame 형태로 변환하여 사용

In [4]:
#Loading datasets
# read dat file
ratings_list = [i.strip().split("::") for i in open(os.path.join(DATA_DIR,'ratings.dat'), 'r').readlines()]
users_list = [i.strip().split("::") for i in open(os.path.join(DATA_DIR,'users.dat'), 'r').readlines()]
movies_list = [i.strip().split("::") for i in open(os.path.join(DATA_DIR,'movies.dat'),encoding='latin-1').readlines()]

# Craete DataFrame
ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = np.uint32)
ratings_df = ratings_df.astype(int).sort_values(["UserID", "Timestamp"])


movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)
users_df = pd.DataFrame(users_list, columns=['UserID','Gender','Age','Occupation','Zip-code'])

#### 다른 추천모델과의 비교를 위해 Train 데이터와 Test 데이터를 구분 (80:20)
NDCG@10, Precision@10을 평가하기 위해 Test 데이터에는 적어도 10개 이상의 데이터를 포함하도록 설정  
(50개 이상의 영화에 대해 피드백을 한 유저들만 추출하여, 시간 순서대로 80:20을 분리하여 데이터 생성)  
* train_data : "valid_ratings_df_train.csv"
* test_data : "valid_ratings_df_test.csv"

In [5]:
ratings_df = pd.read_csv(os.path.join(DATA_DIR,'valid_ratings_df_train.csv'))
test_ratings_df = pd.read_csv(os.path.join(DATA_DIR,'valid_ratings_df_test.csv'))

In [6]:
EMBEDDING_DIM = 100
STATE_SIZE = 10
ACTOR_LEARNIG_RATE = 0.0001
CRITIC_LEARNIG_RATE = 0.0001

log_interval = 100
eval_interval  = 500

NUM_EVAL_EPISODES = 100

REPLAY_BUFFER_MAX_LENGTH = 50000
# NUM_EPISODE = 10000
BATCH_SIZE = 64

### Embedding

Movie-User 간의 Interaction을 고려한 Embedding Network 생성  
학습은 별도의 코드에서 이루어 지므로 여기서는 학습된 네트워크를 load하여 사용

In [7]:
class UserMovieEmbedding(tf.keras.Model):
    def __init__(self, len_users, len_movies, embedding_dim):
        super(UserMovieEmbedding, self).__init__()
        self.m_u_input = tf.keras.layers.InputLayer(name='input_layer', input_shape=(2,))
        # embedding
        self.u_embedding = tf.keras.layers.Embedding(name='user_embedding', input_dim=len_users, output_dim=embedding_dim)
        self.m_embedding = tf.keras.layers.Embedding(name='movie_embedding', input_dim=len_movies, output_dim=embedding_dim)
        # dot product
        self.m_u_merge = tf.keras.layers.Dot(name='movie_user_dot', normalize=False, axes=1)
        # output
        self.m_u_fc = tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, x):
        x = self.m_u_input(x)
        uemb = self.u_embedding(x[0])
        memb = self.m_embedding(x[1])
        m_u = self.m_u_merge([memb, uemb])
        return self.m_u_fc(m_u)

In [8]:
# Embedding Network는 입력되는 item들이 0부터 차례대로 index가 붙어있어야 작동하므로 
# user와 Movie 모두 가지고 있는 데이터의 최대 id값을 활용하여 중간에 id가 건너뛰지 않도록 메꿔주고 임베딩을 한다.
# [0, 1, 5, 6, 10] 과 같이 item 인덱스가 들어가면 나중에 get_layer를 통해 특정 id의 embedding vector를 추출할 때 out of range 에러가 발생할 수 있음
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 으로 만들어준 뒤에 사용
users_num = ratings_df["UserID"].max() + 1
items_num = ratings_df["MovieID"].max() + 1

embedding_network = UserMovieEmbedding(users_num, items_num, EMBEDDING_DIM)
embedding_network([np.zeros((1,)),np.zeros((1,))]) # 아무 값이나 넣어서 네트워크 자체를 한번 Call해야 laod weights를 수행할 수 있음
embedding_network.load_weights('save_weights/user_movie_embedding_case4.h5')

### Environment

#### State Representation Network
논문에 나온 DRR-AVE를 기준으로 생성되었으며, user와 item 간의 interaction을 반영하도록 만들어짐

In [9]:
class DRRAveStateRepresentation(tf.keras.Model):
    def __init__(self, embedding_dim):
        super(DRRAveStateRepresentation, self).__init__()
        self.embedding_dim = embedding_dim
        self.wav = tf.keras.layers.Conv1D(1, 1, 1)
        self.concat = tf.keras.layers.Concatenate()
        self.flatten = tf.keras.layers.Flatten()
        
    def call(self, x):
        items_eb = tf.transpose(x[1], perm=(0,2,1))/self.embedding_dim
        wav = self.wav(items_eb)
        wav = tf.transpose(wav, perm=(0,2,1))
        wav = tf.squeeze(wav, axis=1)
        user_wav = tf.keras.layers.multiply([x[0], wav])
        concat = self.concat([x[0], user_wav, wav])
        return self.flatten(concat)

In [10]:
class RS_Env(py_environment.PyEnvironment):
    def __init__(self, ratings_df, embedding_dim, state_size, embedding_network):
        self.users_num = ratings_df["UserID"].max() + 1
        self.items_num = ratings_df["MovieID"].max() + 1
        self.ratings_df = ratings_df
        self.pos_ratings_df = ratings_df.loc[ratings_df["Rating"] >= 4]
        self.embedding_dim = embedding_dim
        self.embedding_network = embedding_network
        self.state_size = state_size
        self.max_step = 10
        
        # Time Step 내의 항목(action, observation, reward, discount)들의 데이터 크기를 명시해줘야함
        # 환경에서 지정해줘야하는 건 action, observation
        self._action_spec = array_spec.BoundedArraySpec(shape = (embedding_dim, ), dtype = np.float32, maximum = 1, minimum = -1, name = "action")
        self._observation_spec = array_spec.ArraySpec(shape = (3*self.embedding_dim, ), dtype = np.float32, name = "state_representation")
        
        # State Representation을 할 수 있는 사람들을 추출
        # (긍정적인 피드백(4~5점)을 한 영화가 최소 10건 이상인 사람들만 추출)
        self.valid_users = self._generate_valid_user()
        
        # reset env        
        self.reset()
        
        
    def action_spec(self):
        """
            action_spec에 대한 get 함수

            Args: 

            Returns: 
                _action_spec: action_spec(TensorSpec)

            Exception: 
        """        
        return self._action_spec
    
    def observation_spec(self):
        """
            observation_spec 대한 get 함수

            Args: 

            Returns: 
                _observation_spec: observation_spec(TensorSpec)

            Exception: 
        """  
        return self._observation_spec
    
    def _convert_action_score_to_item(self, action_score):
        """
            action_score(actor를 통해 얻어지는 embedding 차원의 벡터)를 후보 item들과의 dot product를 통해 item score로 환산한 뒤
            argmax를 통해 item score가 가장 높은 item을 고르는 함수

            Args: 
                action_score: (actor를 통해 얻어지는 embedding 차원의 벡터 (Numpy.Array)

            Returns: 
                recommendation_item: 추천 영화의 ID(Int)

            Exception: 
        """
        # 전체 item들을 후보군으로 설정한 다음, 이미 추천한 item들을 제외하여 새로운 후보군을 생성.
        items_ids = np.array(range(self.items_num))
        items_ids = np.setdiff1d(items_ids, self.recommended_items)
        
        # embedding network로 부터 후보 item들에 대한 embedding vector를 추출
        items_ebs = self.embedding_network.get_layer('movie_embedding')(items_ids)
        
        
        # 후보 item들의 embedding array와 action_socre를 dot product하여 item_score가 가장 높은 item을 추천
        action_score = tf.convert_to_tensor(np.expand_dims(action_score, 1))
        item_score = tf.keras.backend.dot(items_ebs, action_score)
        item_idx = np.argmax(item_score)
        
        recommendation_item = int(items_ids[item_idx])
        
        return recommendation_item
        
    
    def _reset(self):
        """
            환경을 초기화 시키는 함수
            1) step_count를 1로 초기화하고
            2) valid_users에서 새로운 사람을 random sample하여
            3) 해당 user에 대한 user embedding vector와 state에 대한 item embedding을 추출하여
            4) state representation을 생성

            Args: 

            Returns: 
                ts.restart(self._state): 새로운 state을 넣어 초기화한 time step (time_step)

            Exception: 
        """
        # 1) step_count를 1로 초기화
        self.step_count = 1
        
        # 2) valid_users에서 새로운 사람을 random sample
        
        # valid_users에서 1명의 user를 랜덤 샘플하여 해당 유저의 데이터들을 생성
        # user_df : 해당 유저의 Rating_df 내역
        # state_items_ids : 긍정적인 피드백을 한 영화의 리스트
        self.user_id = np.random.choice(self.valid_users, size = 1).item()
        self.user_df = self.ratings_df.loc[self.ratings_df["UserID"] == self.user_id]

        self.state_items_ids = self.user_df.loc[self.user_df["Rating"] >= 4, "MovieID"].head(self.state_size).values
        self.user_items = self.user_df["MovieID"].values
        
        # 이미 state representation에 사용한 item들(최근 긍정적인 피드백을 한 item들)은 이미 추천한 item리스트에 넣어 추천되지 않도록 함
        self.recommended_items = self.state_items_ids.copy()
        
        # 3) 해당 user에 대한 user embedding vector와 state에 대한 item embedding을 추출
        self.user_eb = self.embedding_network.get_layer('user_embedding')(np.array(self.user_id))
        state_items_eb = self.embedding_network.get_layer('movie_embedding')(np.array(self.state_items_ids))
        
        # 4) state representation을 생성
        self.srm_ave = DRRAveStateRepresentation(self.embedding_dim)
        self._state = self.srm_ave([np.expand_dims(self.user_eb, axis=0), np.expand_dims(state_items_eb, axis=0)])[0]
            
        self._episode_ended = False
        
        return ts.restart(self._state)
        
        
    def _generate_valid_user(self):
        """
            전체 user들 중에서 사용가능한 user들의 list만 추출
            : 긍정적으로 피드백(4이상)한 item을 state_size이상으로 가지고 있는 user들

            Args: 

            Returns: 

            Exception: 
        """
        temp = self.ratings_df.loc[ratings_df["Rating"] >= 4].groupby(["UserID"])["Rating"].count()
        valid_users = temp.loc[temp >= (self.state_size + self.max_step)].index
        
        return valid_users
    
    def _step(self, action):
        """
            Action이 주어졌을 때, 환경이 어떻게 변하는지를 나타내는 step 함수

            Args: 

            Returns: 
                ts.restart(self._state): 새로운 state을 넣어 초기화한 time step (time_step)

            Exception: 
        """
        self.step_count += 1
        
        if self._episode_ended:
            return self.reset()    
        
        # action_score를 기반으로 추천 item 선정
        recommendation_item = self._convert_action_score_to_item(action)
        self.recommendation_item = recommendation_item
        
        # 추천한 item이 유저가 피드백한 item 리스트(user_items)에 있고, 
        #   기존에 추천한 item들(recommended_items)에 없으면,
        # 유저의 피드백에 따라 reward 부여 [-10, 10] (논문과 다름)
        # * reward가 0보다 큰 경우, state을 업데이트.
        if recommendation_item in self.user_items:
            if recommendation_item not in self.recommended_items:
                rate = self.user_df.loc[self.user_df["MovieID"] == recommendation_item, "Rating"].values[0]
                reward = (rate-3)/2 * 10
                if reward > 0:
                    self.state_items_ids = np.append(self.state_items_ids[1:], values = recommendation_item)
                    state_items_eb = self.embedding_network.get_layer('movie_embedding')(np.array(self.state_items_ids))
                    self._state = self.srm_ave([np.expand_dims(self.user_eb, axis=0), np.expand_dims(state_items_eb, axis=0)])[0]
#             else:
#                 reward = 0
        else:
        # 유저가 피드백한 list에 존재하지 않는 경우 -0.1의 reward를 준다.
            reward = -0.1
        
        # 추천한 item은 recommended_items에 넣어서 관리
        # (한번 추천한 item은 재추천하지 않도록)
        self.recommended_items = np.unique(np.append(self.recommended_items, recommendation_item))
        
        
        # max_step에 도달하거나, 유저가 피드백한 item들을 모두 추천한 경우 Episode 종료
        if self.step_count == self.max_step or len(np.setdiff1d(self.user_items, self.recommendation_item)) == 0:
            self._episode_ended = True
        
        
        
        if self._episode_ended:
            return ts.termination(np.array(self._state), reward)
        else:
            return ts.transition(np.array(self._state), reward, discount = 0.9)

In [11]:
# PyEnvironment Instance 생성
train_env_py = RS_Env(ratings_df, embedding_dim = 100, state_size = 10, embedding_network = embedding_network)
eval_env_py = RS_Env(ratings_df, embedding_dim = 100, state_size = 10, embedding_network = embedding_network)

In [12]:
# Wrapper 함수를 활용하여 PyEnvironment를 TFEnvironment로 변환
# TFEnvironment로 변환하면 각각의 time_step 구성 요소들에 차원이 1개씩 추가됨 (배치 차원이 추가되는 것으로 보임)
train_env_tf = tf_py_environment.TFPyEnvironment(train_env_py)
eval_env_tf = tf_py_environment.TFPyEnvironment(eval_env_py)

### Actor Network
DDPG 모듈안에 포함되어 있는 actor_network를 이용하여 ActorNetwork 구현.  
(Layer마다 activation function을 다르게 구현하거나, Conv, RNN, Pooling 등을 구현하려면 별도도 함수를 구현해야함)

In [13]:
from tf_agents.networks import network

In [14]:
actor_net  = tf_agents.agents.ddpg.actor_network.ActorNetwork(
    input_tensor_spec = train_env_tf.observation_spec(), 
    output_tensor_spec = train_env_tf.action_spec(),
    fc_layer_params=[128, 128],
    activation_fn = tf.nn.tanh,
    name = "ActorNetwork"
)

### Critic Network
DDPG 모듈안에 포함되어 있는 critic_network를 이용하여 CriticNetwork 구현.  
(Layer마다 activation function을 다르게 구현하거나, Conv, RNN, Pooling 등을 구현하려면 별도도 함수를 구현해야함)  
ActorNetwork와는 다르게, observation과 action을 각각 Input으로 받기 떄문에 [observation_fc_layer_params]와 같이 각각 의 Input을 전처리하는 Network를 추가할 수 있음

In [17]:
critic_net = tf_agents.agents.ddpg.critic_network.CriticNetwork(
    input_tensor_spec = (train_env_tf.observation_spec(), train_env_tf.action_spec()),
    observation_fc_layer_params = [100],
    joint_fc_layer_params = [128, 128],
    activation_fn = tf.nn.relu,
    output_activation_fn = tf.nn.relu,
    name='CriticNetwork'
)

### Agent
DDPGAgent를 활용하여 구현 (이 부분이 tf_agents를 사용했을 때의 최대의 장점)

In [19]:
train_step = train_utils.create_train_step()

In [20]:
tf_ddpg_agent = tf_agents.agents.DdpgAgent(time_step_spec = train_env_tf.time_step_spec(),
                                           action_spec = train_env_tf.action_spec(),
                                           actor_network = actor_net,
                                           critic_network = critic_net,
                                           actor_optimizer = tf.keras.optimizers.Adam(learning_rate=ACTOR_LEARNIG_RATE),
                                           critic_optimizer = tf.keras.optimizers.Adam(learning_rate=CRITIC_LEARNIG_RATE),
                                           # Target Network를 완전히 별개의 구조로 구현하는 것이 아니라면, 
                                           # 내부적으로 actor network와 critic network를 복제하여 사용
#                                            target_actor_network = target_actor_net, 
#                                            target_critic_network = target_critic_net,
                                           target_update_tau = 0.01,
                                           target_update_period = 1,
                                           gamma = 0.4,
                                           ou_stddev=0.001,
                                           ou_damping=0.01,
                                           train_step_counter = train_step
                                           )

In [21]:
tf_ddpg_agent.initialize()

### Metrics and Evaluation

In [1]:
def compute_avg_return(environment, policy, num_episodes=10):
    """
        환경과 정책이 주어졌을 때, 여러 Episode들을 거치면서 Return의 평균 값을 계산하는 함수
        (여기서 Return은 매 step마다의 reward의 합으로 표현 (Discounted Sum이 아님))
        
        Args: 
            environment: Agent가 상호작용하는 환경 (PyEnvironment or TFEnvironment)
            policy: Agent의 정책
            num_episodes: 한번 평가할 때마다 수행하는 Episode의 수 (Int)

        Returns: 
            Return의 평균 값

        Exception: 
    """
    
    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

In [27]:
from tf_agents.metrics import tf_metrics

In [28]:
ave_return_metric = [tf_agents.metrics.tf_metrics.AverageReturnMetric(
    name='AverageReturn', prefix='Metrics', dtype=tf.float32,
    batch_size=BATCH_SIZE, buffer_size=NUM_EVAL_EPISODES
)]

In [29]:
env_steps = tf_metrics.EnvironmentSteps(prefix='Train')

average_return = tf_metrics.AverageReturnMetric(
    prefix='Train',
    buffer_size=NUM_EVAL_EPISODES,
    batch_size=train_env_tf.batch_size)

train_metrics = [
    tf_metrics.NumberOfEpisodes(prefix='Train'),
    env_steps,
    average_return,
    tf_metrics.AverageEpisodeLengthMetric(
        prefix='Train',
        buffer_size=NUM_EVAL_EPISODES,
        batch_size=train_env_tf.batch_size)]

### Replay Buffer
tf_agents로 PER을 정식적으로 지원하지 않아 Uniform Replay Buffer를 사용  
REPLAY_BUFFER_MAX_LENGTH: 50000

In [31]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(data_spec = tf_ddpg_agent.collect_data_spec,
                                                               batch_size = train_env_tf.batch_size,
                                                               max_length = REPLAY_BUFFER_MAX_LENGTH)

In [33]:
# replay_buffer로 부터 데이터 셋을 sampling 하도록 함 (as_dataset)
# sampling한 데이터 셋을 iter함수를 활용해 iterator롤 만들어서 train 시 사용
dataset = replay_buffer.as_dataset(num_parallel_calls=3, 
                                   sample_batch_size = BATCH_SIZE, 
                                   num_steps=2).prefetch(3)

iterator = iter(dataset)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [3]:
# add_batch를 observer로 설정하여 driver로 agent를 작동시키는 과정에서 지속적으로 데이터를 저장하도록 설정
replay_observer = [replay_buffer.add_batch]

NameError: name 'replay_buffer' is not defined

### Data Collection

In [34]:
from tf_agents.drivers import dynamic_step_driver, dynamic_episode_driver
from tf_agents.policies import random_tf_policy

In [35]:
INITIAL_COLLECT_EPISODES = 10
COLLECT_EPISODES_PER_ITERATION = 1

In [36]:
# 초기 수집 정책(initial_collect_policy)와 수집 정책(collect_policy) 생성
# !!일반 policy를 사용하면 안됌.
initial_collect_policy = random_tf_policy.RandomTFPolicy(train_env_tf.time_step_spec(), train_env_tf.action_spec())
collect_policy = tf_ddpg_agent.collect_policy

In [37]:
# DynamicEpisodeDriver를 활요하여 Episode 단위로 Agent를 동작하도록 설정
# (환경이 초기화 될 떄마다, 새로운 user에 대한 정보를 수집하므로 )
initial_collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(train_env_tf,
                                                                     initial_collect_policy,
                                                                     observers = replay_observer + train_metrics,
                                                                     num_episodes = INITIAL_COLLECT_EPISODES)

collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(train_env_tf,
                                                             collect_policy,
                                                             observers = replay_observer + train_metrics,
                                                             num_episodes = COLLECT_EPISODES_PER_ITERATION)

In [38]:
# def collect_episode(environment, policy, num_episodes):

#     episode_counter = 0
#     traj_counter = 0
#     environment.reset()
# #     print(episode_counter)
#     while episode_counter < num_episodes:
#         time_step = environment.current_time_step()
#         action_step = policy.action(time_step)
#         next_time_step = environment.step(action_step.action)
#         traj = trajectory.from_transition(time_step, action_step, next_time_step)
#         traj_counter += 1
#         # Add trajectory to the replay buffer
        
#         if np.array(traj.reward)[0] != -0.1: 
#             replay_buffer.add_batch(traj)           

#         if traj.is_boundary():
#             episode_counter += 1
#     return traj_counter

### Train

In [40]:
def generate_valid_user(ratings_df, state_size):
    """
        전체 user들 중에서 사용가능한 user들의 list만 추출
        : 긍정적으로 피드백(4이상)한 item을 state_size이상으로 가지고 있는 user들

        Args: 

        Returns:  

        Exception: 
    """
    temp = ratings_df.loc[ratings_df["Rating"] >= 4].groupby(["UserID"])["Rating"].count()
    valid_users = temp.loc[temp >= state_size].index
    

    return ratings_df.loc[ratings_df["UserID"].isin(valid_users)]

In [41]:
def recommend_random_item(recommended_items, embedding_network, top_k = 1):
    """
        임의의 item을 추천하도록 하는 함수
        (모델 성능 비교 용)
        
        Args: 
            recommended_items: 기존에 추천한 item 리스트 
            embedding_network: 추천한 item들의 embedding array를 추출하기 위한 embedding network
            top_k: 추천할 item의 개수 (Int)

        Returns:  
            랜덤으로 추출한 k개의 추천 item Array

        Exception: 
    """
    item_num = embedding_network.get_layer("movie_embedding").get_weights()[0].shape[0]
    items_ids = np.array(range(items_num))

    items_ids = np.setdiff1d(items_ids, recommended_items)

    return np.random.choice(items_ids, 10)

In [4]:
def convert_action_score_item(action_score, recommended_items, embedding_network, top_k = 1):
    """
        action_score(actor를 통해 얻어지는 embedding 차원의 벡터)를 후보 item들과의 dot product를 통해 item score로 환산한 뒤,
        argmax를 통해 item score가 가장 높은 item을 고르는 함수
        * 평가용 함수
          Tensor로 반환되므로 로직자체는 동일하지만 내부 연산과정이 RSEnv내의 메서드와 약간 다름

        Args: 
            action_score: (actor를 통해 얻어지는 embedding 차원의 텐서 (Tensor)
            recommended_items: 이미 추천된 item의 list (list)
            embedding_network: embedding array를 추출하기 위한 embedding network
            top_k: 추천할 item의 개수 (Int)

        Returns: 
            recommendation_item: 추천 영화의 ID(Int)

        Exception: 
    """
    item_num = embedding_network.get_layer("movie_embedding").get_weights()[0].shape[0]
    items_ids = np.array(range(items_num))

    items_ids = np.setdiff1d(items_ids, recommended_items)

    items_ebs = embedding_network.get_layer('movie_embedding')(items_ids)
    action_score = tf.transpose(action_score, perm=(1,0))
#     action_score = tf.convert_to_tensor(np.expand_dims(action_score, 1))

    item_score = tf.keras.backend.dot(items_ebs, action_score)
    item_score = np.array(item_score).reshape((item_score.shape[0],))
    
    recommendation_items = item_score.argsort()[::-1][:10]

    return recommendation_items

In [43]:
def calculate_ndcg(rel, irel):
    """
        NDCG를 산출하는 함수
        
        Args: 
            rel: 추천한 item이 정답지에 존재하면 1, 아니면 0으로 분류된 list
            irel: 이상적인 리스트로 모두가 1(정답지에 존재)인 list

        Returns:  
             NDCG 값 (dcg/idcg)

        Exception: 
    """
    dcg = 0
    idcg = 0
    rel = [1 if r>0 else 0 for r in rel]
    for i, (r, ir) in enumerate(zip(rel, irel)):
        dcg += (r)/np.log2(i+2)
        idcg += (ir)/np.log2(i+2)
    return dcg/idcg

In [44]:
def create_correct_list(test_user_df, recommendation_list):
    """
        추천한 item이 정답지(긍정적인 피드백을 한 목록)에 존재하면 1, 아니면 0으로 라벨링된 List를 생성하는 함수
        (NDCG 계산에 사용)
        
        Args: 
            test_user_df: 추천한 item이 정답지에 존재하면 1, 아니면 0으로 분류된 list
            recommendation_list: 추천된 item의 list (list)

        Returns:  
             NDCG 값 (dcg/idcg)

        Exception: 
    """
    result = []
    for x in recommendation_list:
        if x in test_user_df["MovieID"].values:
            if int(test_user_df.loc[test_user_df["MovieID"] == x, "Rating"].values) >= 4:
                result.append(1)
        else:
            result.append(0)
    return result

In [45]:
def evaluate_model(ratings_df, test_ratings_df, policy, embedding_network, embedding_dim, state_size, top_k):
    """
        Agent의 정책을 평가하는 함수
        test_rating_df에 포함된 user들 마다의 ndcg, precision을 구하여 list로 반환
        (학습 과정에서 가장 마지막 10개의 긍정적인 피드백을 한 item들을 State으로 가정하고,
         추천한 k 개의 item이 test 셋에 존재하는 지를 평가)
        
        Args: 
            ratings_df: 학습에 사용하는 데이터 셋 (Pandas.DataFrame)
            test_ratings_df: 평가에 사용하는 데이터 셋(Pandas.DataFrame)
            policy: 학습된 Agent의 Actor 정책 
            embedding_network: embedding array를 추출하기 위한 embedding network
            embedding_dim: embedding 차원 (Int)
            state_size: State에 사용하는 item의 수 (Int)
            top_k: 추천할 item의 개수 (Int)

        Returns:  
             ndcg_list, precision_list

        Exception: 
    """
    # 학습 데이터에서 긍정적인 피드백을 한 item이 10개 이상인 user의 데이터만 추출
    valid_ratings_df = generate_valid_user(ratings_df, 10)
    
    ndcg_list = []
    precision_list = []
    for idx, user_id in enumerate(valid_ratings_df["UserID"].unique()):
        
        # 학습 과정에서 가장 마지막 10개의 긍정적인 피드백을 한 item들을 State으로 설정
        user_df = valid_ratings_df.loc[valid_ratings_df["UserID"] == user_id]
        state_items_ids = user_df.loc[user_df["Rating"] >= 4, "MovieID"].tail(state_size).values
        user_items = user_df["MovieID"].values
        
        # state에 사용된 item들은 추천되지 않도록 설정
        # ???? Train에 사용된 모든 item들을 추천되지 않도록 설정해야하나?
        recommended_items = state_items_ids.copy()

        # 해당 user에 대한 user embedding vector와 state에 대한 item embedding을 추출
        user_eb = embedding_network.get_layer('user_embedding')(np.array(user_id))
        state_items_eb = embedding_network.get_layer('movie_embedding')(np.array(state_items_ids))

        # state representation을 생성
        srm_ave = DRRAveStateRepresentation(embedding_dim)
        _state = srm_ave([np.expand_dims(user_eb, axis=0), np.expand_dims(state_items_eb, axis=0)])[0]
        
        # policy.action에 넣기 위해서는 time_step 형태가 되어야 하므로, state_representation 값으로 observation을 대체하여 time_step을 생성
        input_step = tf_agents.trajectories.TimeStep(step_type = 1, reward = 0, discount = 0.9, observation = tf.expand_dims(_state,0))
        
        # Action_score 산출하여 k개의 추천 item을 선정
        action_score = policy.action(input_step).action
        recommendation_list = convert_action_score_item(action_score = action_score, recommended_items = recommended_items, embedding_network = embedding_network, top_k = top_k)

        # Test 데이터에 존재하는 item들과의 비교를 통해 NDCG와 Precision을 산출
        test_user_df = test_ratings_df.loc[test_ratings_df["UserID"] == user_id]
        correct_list = create_correct_list(test_user_df, recommendation_list)
        ndcg = calculate_ndcg(correct_list, [1 for _ in range(len(correct_list))])

        ndcg_list.append(ndcg)
        precision_list.append(sum(correct_list)/len(correct_list))

    return ndcg_list, precision_list

In [47]:
initial_collect_driver.run = common.function(initial_collect_driver.run)
collect_driver.run = common.function(collect_driver.run)
tf_ddpg_agent.train = common.function(tf_ddpg_agent.train)

In [48]:
# Collect initial replay data.
# 한번도 데이터 수집 환경의 step이 진행되지 않았거나, replay_buffer에 쌓여있는 데이터의 수가 0인 경우
# Inital collect_step을 수행
if env_steps.result() == 0 or replay_buffer.num_frames() == 0:
    logging.info(
        f"Initializing replay buffer by collecting experience for {INITIAL_COLLECT_EPISODES} steps with a random policy.")
    initial_collect_driver.run()

In [52]:
NUM_EPISODE = 1000000

LOG_INTERVAL = 100
EVAL_INTERVAL = 500

NUM_EVAL_EPISODES = 100
TOP_K = 10

In [49]:
# time_step과 policy_state 초기화
time_step = None
policy_state = collect_policy.get_initial_state(train_env_tf.batch_size)

In [26]:
# avg_return 값을 저장해둘 List 초기화
returns = []
returns.append(compute_avg_return(eval_env_tf, tf_ddpg_agent.policy, NUM_EVAL_EPISODES))

In [53]:
for _ in range(NUM_EPISODE):
    # 한번의 loop 마다 1개의 Episode(user 1명에 대한 Trajectory)의 Data를 수집
    time_step, policy_state = collect_driver.run(time_step = time_step,
                                                 policy_state = policy_state)
    
    # Replay Buffer로 부터 학습용 데이터 추출
    experience, unused_info = next(iterator)
    
    # 학습(actor network, critic_network)
    train_loss = tf_ddpg_agent.train(experience)

    step = tf_ddpg_agent.train_step_counter.numpy()
    
    if step % LOG_INTERVAL == 0:
        print(f'step = {step}: actor_loss = {np.array(train_loss[1].actor_loss)}, critic_loss = {np.array(train_loss[1].critic_loss)}')
    
    # 평가용 Environment 에서 100번의 Episode를 거치면서 평균 Return (avg_return)을 산출
    # 해당 값은 returns 리스트에 저장
    if step % EVAL_INTERVAL == 0:
        avg_return = compute_avg_return(eval_env_tf, tf_ddpg_agent.policy, NUM_EVAL_EPISODES)
        print(f'Evaluation Average Return = {avg_return}')
        returns.append(avg_return)
    
    if step % 10000 == 0:
        ndcg, precision = evaluate_model(ratings_df, test_ratings_df, tf_ddpg_agent.policy, embedding_network, EMBEDDING_DIM, STATE_SIZE, TOP_K)
        print(f"ndcg@{TOP_K} : {np.mean(ndcg)}, precision@{TOP_K} : {np.mean(precision)}")

step = 100: actor_loss = 2.573166000274796e-07, critic_loss = 0.14485901594161987
step = 200: actor_loss = 3.8035861962271156e-06, critic_loss = 0.7275145053863525
step = 300: actor_loss = 1.3831613614456728e-05, critic_loss = 0.2816231846809387
step = 400: actor_loss = 3.7447964132297784e-05, critic_loss = 0.787411630153656
step = 500: actor_loss = 3.9788024878362194e-05, critic_loss = 0.7971856594085693
Evaluation Average Return = 4.27700138092041
step = 600: actor_loss = 6.55280236969702e-05, critic_loss = 0.4809677302837372
step = 700: actor_loss = 4.948253263137303e-05, critic_loss = 0.7275082468986511
step = 800: actor_loss = 5.350480932975188e-05, critic_loss = 0.6057593822479248
step = 900: actor_loss = 5.009876622352749e-05, critic_loss = 0.9335842132568359
step = 1000: actor_loss = 6.850966019555926e-05, critic_loss = 1.08821702003479
Evaluation Average Return = 10.178000450134277
step = 1100: actor_loss = 9.117728041019291e-05, critic_loss = 1.0628429651260376
step = 1200: a

### Model Save

In [59]:
from tf_agents.policies import policy_saver

In [60]:
policy_dir = os.path.join(ROOT_DIR, 'policy_210726_1M')
tf_policy_saver = PolicySaver(tf_ddpg_agent.policy)

In [61]:
tf_policy_saver.save(policy_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /home/jovyan/RSRL_TF_Agent/policy_210723_1M/assets


INFO:tensorflow:Assets written to: /home/jovyan/RSRL_TF_Agent/policy_210723_1M/assets
